In [1]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fernandoleonfranco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandoleonfranco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
os.chdir(r'/Users/fernandoleonfranco/Documents/GitHub/Semestre_III/DataIntelligence/exist2021')
print(os.getcwd())

/Users/fernandoleonfranco/Documents/GitHub/Semestre_III/DataIntelligence/exist2021


In [3]:
# Cargar los datos
df_train = pd.read_csv("train.tsv", sep='\t')
df_test = pd.read_csv("test.tsv", sep='\t')

print('Tamaño training:', df_train.shape)
print('Tamaño test:', df_test.shape)

Tamaño training: (6977, 7)
Tamaño test: (4368, 7)


In [4]:
# Filtrar por idioma inglés
df_train = df_train[df_train['language'] == 'en']
df_test = df_test[df_test['language'] == 'en']
print('Tamaño training (solo inglés):', df_train.shape)
print('Tamaño test (solo inglés):', df_test.shape)

Tamaño training (solo inglés): (3436, 7)
Tamaño test (solo inglés): (2208, 7)


In [5]:
# Preparar las columnas necesarias
df_train = df_train.drop(['test_case', 'id', 'source', 'task2'], axis=1)
df_test = df_test.drop(['test_case', 'id', 'source', 'task2'], axis=1)
df_train = df_train.rename(columns={'task1': 'label'})
df_test = df_test.rename(columns={'task1': 'label'})
print('Primeros registros del training:')
print(df_train.head())

Primeros registros del training:
  language                                               text       label
0       en  She calls herself "anti-feminazi" how about sh...      sexist
1       en  Now, back to these women, the brave and the be...  non-sexist
2       en  @CurvyBandida @Xalynne_B Wow, your skirt is ve...      sexist
3       en  @AurelieGuiboud Incredible!  Beautiful!But I l...  non-sexist
4       en  i find it extremely hard to believe that kelly...  non-sexist


In [6]:
stopwords_en = stopwords.words("english")

# Preprocesamiento de texto
def clean_text(text):
    text = str(text).lower()  # Minúsculas
    from nltk.tokenize.punkt import PunktSentenceTokenizer
    punkt_tokenizer = PunktSentenceTokenizer()  # Instanciar el tokenizador
    
    tokens = punkt_tokenizer.tokenize(text)  # Tokenizar
    tokens = [word for word in tokens if word not in stopwords_en]  # Eliminar stopwords
    tokens = [PorterStemmer().stem(word) for word in tokens]  # Stemming
    min_length = 3
    p = re.compile('^[a-zA-Z]+$')
    filtered_tokens = [token for token in tokens if len(token) >= min_length and p.match(token)]
    return filtered_tokens